### Modelo **user-items**

In [6]:
import pandas as pd
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import Dataset, Reader, SVD 
import numpy as np
import pandas as pd
from joblib import dump

In [14]:
df_modelo = pd.read_parquet('Dict\df_final.parquet')

In [12]:
df_modelo.count()

user_id               510027
item_id               510027
item_name             510027
playtime_forever      510027
recommend             510027
year                  510027
sentiment_analysis    510027
genres                510027
release_date          510027
price                 510027
developer             510027
rating                510027
dtype: int64

In [5]:
# Tomar una muestra del 10% de los datos
df_modelo = df_modelo.sample(frac=0.1, random_state=42)

# Reemplazar los valores de la columna 'recommend'
df_modelo['recommend'] = df_modelo['recommend'].replace({True: 1, False: 0})

# Crear la columna 'rating'
df_modelo['rating'] = 0
df_modelo.loc[(df_modelo['sentiment_analysis'] == 2) & (df_modelo['recommend'] == 1), 'rating'] = 5
df_modelo.loc[(df_modelo['sentiment_analysis'] == 2) & (df_modelo['recommend'] == 0), 'rating'] = 4
df_modelo.loc[(df_modelo['sentiment_analysis'] == 1) & (df_modelo['recommend'] == 1), 'rating'] = 3
df_modelo.loc[(df_modelo['sentiment_analysis'] == 1) & (df_modelo['recommend'] == 0), 'rating'] = 2
df_modelo.loc[(df_modelo['sentiment_analysis'] == 0) & (df_modelo['recommend'] == 1), 'rating'] = 1

# Guardar el DataFrame en un nuevo archivo .parquet
df_modelo.to_parquet('user_item_modelo.parquet')

# Cargar los datos del archivo .parquet
new_df = pd.read_parquet('user_item_modelo.parquet')

# Adaptar rating_scale a la escala de datos
reader = Reader(rating_scale=(0, 5))

# Cargar los datos en un objeto Dataset de Surprise
data = Dataset.load_from_df(new_df[['user_id', 'item_name', 'rating']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Optimizar los hiperparámetros con GridSearchCV
param_grid = {'n_factors': [5, 50, 100], 'n_epochs': [5, 10, 20], 'lr_all': [0.001, 0.002, 0.005], 'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs.fit(data)

# Entrenar el modelo SVD con los hiperparámetros optimizados
model = SVD(**gs.best_params['rmse'])
model.fit(trainset)

# Guardar el modelo en un archivo joblib
dump(model, 'user_item_model.joblib')

C:\Users\57301\AppData\Local\Temp\ipykernel_8540\3933399594.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_modelo['recommend'] = df_modelo['recommend'].replace({True: 1, False: 0})


['user_item_model.joblib']

In [28]:
from joblib import load

# cargar el modelo joblib
modelo = load('user_item_model.joblib')

In [31]:
# SISTEMA DE RECOMENDACIÓN - USER-ITEM
def recomendacion_usuario(user_id):
    """
    Sistema de recomendación user-item.

    Ingresando el ID de un usuario, se devuelve una lista con 5 juegos recomendados para dicho usuario.

    Args:
    - user_id (str): El ID del usuario para el cual se desean obtener recomendaciones.

    Returns:
    - dict: Un diccionario que contiene una lista de 5 juegos recomendados para el usuario ingresado.
      El diccionario tiene el formato {"Juegos recomendados para el usuario": user_id,
      'Juego 1': 'Nombre1', 'Juego 2': 'Nombre2', 'Juego 3': 'Nombre3', 'Juego 4': 'Nombre4', 'Juego 5': 'Nombre5'}.

    Raises:
    - Si el DataFrame está vacío, devuelve un mensaje de error.
    - Si el user_id no existe en el DataFrame, devuelve un mensaje de error.
    """
    # # Cargar el modelo del archivo .pkl
    # with open('user_item_model.pkl', 'rb') as archivo:
    #     modelo = pickle.load(archivo)

    # Cargar los datos del archivo .parquet
    new_df = pd.read_parquet('user_item_modelo.parquet')

    # cargar el modelo joblib
    modelo = load('user_item_model.joblib')

    # Si el dataframe está vacío, devolver un mensaje de error
    if new_df.empty:
        return {
        "detail": [
            {
                "loc": ["string", 0],
                "msg": "El DataFrame está vacío",
                "type": "value_error"
            }
        ]
    }

    if user_id not in new_df['user_id'].unique():
        return {
        "detail": [
            {
                "loc": ["string", 0],
                "msg": f"El usuario {user_id} no existe.",
                "type": "value_error"
            }
        ]
    }

    juegos_valorados = new_df[new_df['user_id'] == user_id]['item_name'].unique()
    todos_los_juegos = new_df['item_name'].unique()
    juegos_no_valorados = list(set(todos_los_juegos) - set(juegos_valorados))
    predicciones = [modelo.predict(user_id, juego) for juego in juegos_no_valorados]
    recomendaciones = sorted(predicciones, key=lambda x: x.est, reverse=True)[:5]
    juegos_recomendados = [recomendacion.iid for recomendacion in recomendaciones]
    recomendaciones_dict = {"Juegos recomendados para el usuario": user_id, 'Juego 1': juegos_recomendados[0], 'Juego 2': juegos_recomendados[1], 'Juego 3': juegos_recomendados[2], 'Juego 4': juegos_recomendados[3], 'Juego 5': juegos_recomendados[4]}
    return recomendaciones_dict

In [44]:
# llamar a la funcion
recomendacion_usuario('tru_mu_')

{'Juegos recomendados para el usuario': 'tru_mu_',
 'Juego 1': 'Deus Ex: Mankind Dividedâ\x84¢',
 'Juego 2': 'Project Zomboid',
 'Juego 3': "Tom Clancy's The Division",
 'Juego 4': 'Magic 2014 ',
 'Juego 5': 'Youtubers Life'}

### Modelo de recomendación **item-item**

Aplicamos similitud del coseno

![similitud coseno](img\similitud_coseno.PNG)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from joblib import dump
import ast

# Convertir la columna de géneros a una lista de géneros
# df_modelo['genres'] = df_modelo['genres'].apply(ast.literal_eval)

# Agrupar por item_id y combinar los géneros en una lista única
df_grouped = df_modelo.groupby('item_id')['genres'].sum().reset_index()

# Crear el codificador One-Hot
mlb = MultiLabelBinarizer()

# Aplicar el codificador One-Hot a la columna de géneros
one_hot_genres = mlb.fit_transform(df_grouped['genres'])

# Crear un DataFrame con las columnas codificadas
df_genres = pd.DataFrame(one_hot_genres, columns=mlb.classes_, index=df_grouped['item_id'])

# Calcular la similitud del coseno entre los ítems
item_similarity = cosine_similarity(df_genres)

# Convertir el resultado en un DataFrame para facilitar su manejo
item_similarity_df = pd.DataFrame(item_similarity, index=df_genres.index, columns=df_genres.index)

# Guardar el DataFrame en un archivo parquet
dump(item_similarity_df, "item_similarity_df.joblib")

In [ ]:
from joblib import load
# SISTEMA DE RECOMENDACIÓN - ITEM-ITEM
def recomendacion_juego(item_id):
    """
    Sistema de recomendación item-item.

    Ingresando el ID de un producto, se devuelve una lista con 5 juegos recomendados similares al ingresado.

    Args:
    - item_id: El ID del producto para el cual se desean obtener recomendaciones.

    Returns:
    - dict: Un diccionario que contiene una lista de 5 juegos recomendados similares al ingresado.
      El diccionario tiene el formato {"Juegos similares al ID ingresado": {ID1: 'Nombre1', ID2: 'Nombre2', ...}}.

    Raises:
    - Si el DataFrame está vacío, devuelve un mensaje de error.
    - Si el item_id no existe en el DataFrame, devuelve un mensaje de error.
    - Si ocurre un error al cargar el archivo de similitud, intenta calcular la similitud del coseno
      entre los productos y proporciona recomendaciones basadas en eso.
    """
    # Leer el archivo parquet en un DataFrame
    data = df_modelo

    try:
        # Cargar la matriz de similitud de items_similarity
        item_similarity_df = load('/content/items_similar.joblib')

        # mensaje de test para verificar que se cargó la matriz precalculada
        print("Se cargó la matriz precalculada.")

    except Exception as e:
        print(f"Ocurrió un error al cargar el archivo: {e}")

    # Si el dataframe está vacío, devolver un mensaje de error
    if data.empty:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El DataFrame está vacío",
                    "type": "value_error"
                }
            ]
        }

    # Si el item_id no existe en el DataFrame, devolver un mensaje de error
    if item_id not in data['item_id'].values:
        return {
            "detail": [
                {
                    "loc": ["string", 0],
                    "msg": "El item_id proporcionado no existe",
                    "type": "value_error"
                }
            ]
        }

    # Obtener los juegos similares al juego dado
    similar_games = item_similarity_df[item_id].sort_values(ascending=False)

    # Obtener los 5 juegos más similares
    similar_games_ids = similar_games.head(6).index.tolist()[1:]  # Excluimos el primer juego porque es el mismo juego

    # Crear un diccionario con los ids y los nombres de los juegos
    recommended_games = {game_id: data[data['item_id'] == game_id]['item_name'].iloc[0] for game_id in similar_games_ids}

    return {"Juegos similares al id ingresado": recommended_games}

In [ ]:
# llamado a la función
recomendacion_juego(333930)